# Statefarm Data - Phase5 - Final Modeling Using Validation Data

Comparing various models after removal of marginal quality data and using 14000 cases of pseudo labeled data

In [1]:
import theano
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
IMPORT_DIR = '/home/ubuntu/nbs'
%cd $IMPORT_DIR

/home/ubuntu/nbs


In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import daveutils
from daveutils import *
import davenet
from davenet import *
import my_cv_modeler
from my_cv_modeler import *

Using Theano backend.


In [4]:
ALL_DATA_DIR = '/home/ubuntu/'
DATA_HOME_DIR = ALL_DATA_DIR+'statefarm/'
TRAIN_DIR = DATA_HOME_DIR+'train/'
VALID_DIR = DATA_HOME_DIR+'valid/'
SAMPLE_DIR = DATA_HOME_DIR+'sample/'
MODELS_DIR = DATA_HOME_DIR+'models/'
RESULTS_DIR = DATA_HOME_DIR+'results/'
TEST_DIR = DATA_HOME_DIR+'test/'

# 1. Prepare Data

#### Identify and remove poor quality training data

Previously Identified Data that is badly classified or multi-class:

In [5]:
%cd $DATA_HOME_DIR

/home/ubuntu/statefarm


In [32]:
ALL_DATA_DIR = '/home/ubuntu/'
TRAIN_DIR = ALL_DATA_DIR+'statefarm/train' # yes, this still includes the pseudo labelled data
VALID_DIR = ALL_DATA_DIR+'statefarm/valid' #nb Notice that I've gone back to the orginal directory here
from shutil import copy
#%cd $DATA_HOME_DIR
def copyFromValidToTrain():  #bad_dir must not already exist
    count = 0
    g = glob(VALID_DIR+'/c?/*.jpg')
    for filename in g:
        #print(TRAIN_DIR+filename[28:])
        copy(filename, TRAIN_DIR+filename[28:])
        count+=1
    print(count,"items successfully copied from ",VALID_DIR,"folder to: ",TRAIN_DIR)

In [33]:
copyFromValidToTrain()

3827 items successfully copied from  /home/ubuntu/statefarm/valid folder to:  /home/ubuntu/statefarm/train


# 2. Reload our previous best Sequential Vgg16 Model 

In [6]:
vgg = Dave16()

In [9]:
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]
count_frozen = 0
for layer in conv_layers:
    layer.trainable = False
    if layer.trainable == False: count_frozen+=1
print(count_frozen,"layers are frozen")  
conv_model = Sequential(conv_layers)
top_hat_model = read_model(4, cross='old') 

31 layers are frozen


In [10]:

def add_bn_layers(p, model):
    new_model = model
    new_model.add(MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]))
    new_model.add(Flatten())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[3].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p/2))
    new_model.add(Dense(128, activation='relu'))
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[6].get_weights())
    new_model.add(BatchNormalization())
    new_model.add(Dropout(p))
    new_model.add(Dense(10, activation='softmax'))  
    #new_model.layers[len(new_model.layers)].set_weights(top.layers[9].get_weights())
    return new_model

In [11]:
full_model = add_bn_layers(0.5, conv_model)


In [ ]:
full_model.layers[last_conv_idx+3+1].set_weights(top_hat_model.layers[3].get_weights())
full_model.layers[last_conv_idx+6+1].set_weights(top_hat_model.layers[6].get_weights())
full_model.layers[last_conv_idx+9+1].set_weights(top_hat_model.layers[9].get_weights())

In [19]:
full_model.load_weights('/home/ubuntu/statefarm/cache/model_weights1vgg_minus_val.h5')

In [20]:
full_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 3. Train the Model - including use of 14k pseudo label test cases

n.b. Mixiterator was not used.  Only test data having a prediction probability >0.995 has been used.
This data is considered to be of such good quality that it can be mixed with real data. The pseudo training data will make up 43% of the training data at this stage (39% after validation data is added). Yes, it's a little high, but lets see how it goes.. 

Create the image generator (no augmentation)

In [14]:
gen = ImageDataGenerator()

In [15]:
val_generator = gen.flow_from_directory(
        'valid',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)
val_generator.N

Found 3827 images belonging to 10 classes.


3827

## Adding Data Augmentation

In [34]:
dgen = ImageDataGenerator(  rotation_range=5,
                            width_shift_range=0.1,
                            height_shift_range=0.05,
                            channel_shift_range = 20
                         )
tgenerator = dgen.flow_from_directory(
        'train',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)


Found 36648 images belonging to 10 classes.


In [35]:
full_model.optimizer.lr=0.00005
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=500)

Epoch 1/1
36648/36648 [==============================] - 955s - loss: 0.1099 - acc: 0.9719 - val_loss: 0.3268 - val_acc: 0.8848


In [36]:
save_model(full_model, 1, cross='vgg16final')

In [37]:
full_model.optimizer.lr=0.00001
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
36648/36648 [==============================] - 1039s - loss: 0.0729 - acc: 0.9784 - val_loss: 0.1808 - val_acc: 0.9540


In [38]:
save_model(full_model, 2, cross='vgg16final')

In [39]:
full_model.optimizer.lr=0.00001
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
36648/36648 [==============================] - 1041s - loss: 0.0581 - acc: 0.9820 - val_loss: 0.1180 - val_acc: 0.9705


In [40]:
save_model(full_model, 3, cross='vgg16final')

In [41]:
full_model.optimizer.lr=0.00001
full_model.fit_generator(
        tgenerator,
        samples_per_epoch=tgenerator.N,
        nb_epoch=1,
        validation_data=val_generator,
        nb_val_samples=val_generator.N)

Epoch 1/1
36648/36648 [==============================] - 1039s - loss: 0.0525 - acc: 0.9843 - val_loss: 0.0936 - val_acc: 0.9733


In [42]:
save_model(full_model, 4, cross='vgg16final')

# Conclusion

In conclusion, the above validation dataset looks highly accurate, but do not be mislead, there same subject appear in the training dataset and validation dataset, albiet the versions in the training dataset are augmented.  

Pose estimation is difficult when harnessing transfer learning from Vgg16 (or Vgg19) because different poses were not different classes in the original Vgg modeles, nor were hands, hands holding cups, hand holding mobile phone down low, hands holding mobile phone up near ears. 22k images is hardly sufficient training data for classifying vastly similar poses in 224x224 images with vastly different test subjects; especially when bounding boxes are not provided; and especially when there are 10 different classes hence a random uniform guess at the actual class is only expected to be correct around 10% of the time, i.e. there is much bigger gap (to 100% c.c.r.) as compared to a two-class classifier.  

The next logical step is to incorporate hand, steering wheel, face, phone, make-up mirror/gaze direction bounding boxes (or segmentation) into a multi-label neural network model to improve performance.  For example, a model  bounding boxes for hands could be trained on an annotated hand dataset (e.g. http://www.robots.ox.ac.uk/~vgg/data/hands/) then used to predict the bounding boxes for hands in the images of the distracted-driver data set.  Similarly, the bounding boxes for steering wheel, face, phone, make-up mirror and gaze direction (can just be a two points forming a vector from the estimated centre of the eyeball to the centre of the pupil). Using the functional model API of Keras, the outputs of the bounding boxes can be connected with regression activation function to the second last layer of the model for the classification output.